In [1]:
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
from project.ship import Ship
from project.tests import Tests
from project.botma import TMA
import time
from pathlib import Path
from datetime import datetime

In [2]:
# Класс для сохранения результатов
tests = Tests('test1')
Path("../tests/test1/d40").mkdir(parents=True, exist_ok=True)
print("Время последнего моделирования:", datetime.now().strftime("%d/%m/%Y %H:%M"))

# Создаем наблюдателя
observer_x, observer_y, observer_course, observer_velocity = 0, 0, 0, 5
observer = Ship('Наблюдатель', observer_x, observer_y, observer_course,
                observer_velocity, verbose=True)

Время последнего моделирования: 26/03/2021 04:32


In [3]:
# Моделирование траектории наблюдателя
observer.forward_movement(3 * 60)
observer.change_course(270, 'left', omega=0.5)
observer.forward_movement(5 * 60)
observer.change_course(90, 'right', omega=0.5)
observer.forward_movement(3 * 60)

Наблюдатель движется прямо по курсу 0.0° 180с
Наблюдатель перешёл на курс 270.0° за 180с
Наблюдатель движется прямо по курсу 270.0° 300с
Наблюдатель перешёл на курс 90.0° за 360с
Наблюдатель движется прямо по курсу 90.0° 180с


In [4]:
def target_func(seed=None):
    np.random.seed(seed)
    b = 0
    d = 40
    c = np.random.uniform(0, 180)
    v = np.random.uniform(3, 15)
    return [b, d, c, v]
results = []
p0 = [0., 25., 90., 7.]
std_arr = [0., 0.1, 0.2, 0.3, 0.5, 1.]
precisions_1 = ['{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}',
'{0:.2f}','{0:.3f}', '{0:.2f}','{0:.3f}','{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{:.0f}', '{0:.4f}', '{0:.0f}', '{0:.0f}']
precisions_2 = ['{0:.1f}', '{0:.3f}', '{0:.1f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.1f}', '{0:.2f}', '{0:.2f}','{0:.3f}', '{0:.2f}','{0:.3f}', '{0:.2f}', '{0:.2f}', '{0:.0f}', '{0:.4f}']
precisions_3 = ['{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.2f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.0f}', '{0:.4f}', '{0:.4f}', '{0:.1f}']

In [5]:
for std in std_arr:

    tma = TMA(observer, std=np.radians(std), tau=2, verbose=True)
    algorithm_name = 'ММП v2'
    res = [algorithm_name]
    dict_results = tma.swarm(algorithm_name = algorithm_name, n=100, target_func=target_func, p0=p0, seeded=True, verbose=True)
    df = tests.get_df(dict_results)

    print('Показатели Монте-Карло:')
    for i in range(4):
        success = df['Успех'].apply(lambda x: x[i])
        success_rate = 100 * success.mean()
        success_min = min(success)
        print('Вариант {}: Рэф = {:.0f}%, Рэф_мин = {:.0f}'.format(i, success_rate, success_min))
        res.extend([success_rate, success_min])
    k_a_mean = df['Ка'].mean()
    nf_mean = df['Nf'].mean()
    nf_max = max(df['Nf'])
    iter_mean = df['Iter'].mean()
    iter_max = max(df['Iter'])
    t_mean = df['t'].mean()
    t_max = max(df['t'])
    std = np.degrees(tma.standart_deviation)
    print('Ка = {:.2f}'.format(k_a_mean))
    print('Nf = {:.0f}, Nf_max = {:.0f}'.format(nf_mean, nf_max))
    print('Iter = {:.0f}, Iter_max = {:.0f}'.format(iter_mean, iter_max))
    print('t = {:.4f}, t_max = {:.4f}'.format(t_mean, t_max))

    d_df = df.copy()
    for i, j in enumerate(df.columns):
        try: d_df[j] = df[j].apply(lambda x: precisions_1[i].format(np.float64(x)))
        except(TypeError): pass
    display(d_df.drop(['СКО X', 'СКО Y', 'СКО VX', 'СКО VY', 'Кб', 'П0_апр', 'Д0_апр', 'К0_апр', 'V0_апр'], axis=1).head())

    res.extend([k_a_mean, nf_mean, nf_max, iter_mean, iter_max, t_mean, t_max])
    results.append(res)
    tests.save_df(df, name='../tests/test1/d40/ЛМ, ' + str(std) + '.xlsx')
    print('_'*40)

df_lm = pd.DataFrame(results, columns=['Алгоритм', 'В11', 'В12', 'В21', 'В22', 'В31', 'В32', 'В41', 'В42', 'Ка', 'Nf', 'Nf_max', 'Iter', 'Iter_max', 't', 't_max'])
df_lm['СКО'] = std_arr
results = []

СКОп = 0.0, tau = 2, end_time = 1200
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.5 с
Показатели Монте-Карло:
Вариант 0: Рэф = 100%, Рэф_мин = 1
Вариант 1: Рэф = 100%, Рэф_мин = 1
Вариант 2: Рэф = 100%, Рэф_мин = 1
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.00
Nf = 9, Nf_max = 11
Iter = 8, Iter_max = 10
t = 0.0019, t_max = 0.0043


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t,Nf,Iter
0,0.0,40.000,98.786,11.58,0.0,40.000,98.787,11.58,22.92,38.282,22.92,38.281,0.00,"[1, 1, 1, 1]",0.0027,10,9
1,0.0,40.000,75.064,11.64,0.0,40.000,75.064,11.64,19.70,43.532,19.70,43.531,0.00,"[1, 1, 1, 1]",0.0022,10,9
2,0.0,40.000,78.479,3.31,360.0,40.000,78.478,3.31,7.56,38.512,7.56,38.512,0.00,"[1, 1, 1, 1]",0.0024,8,7
3,0.0,40.000,99.144,11.50,0.0,40.000,99.144,11.50,22.80,38.174,22.80,38.173,0.00,"[1, 1, 1, 1]",0.0032,10,9
4,0.0,40.000,174.065,9.57,360.0,40.000,174.065,9.57,5.19,26.072,5.19,26.072,0.00,"[1, 1, 1, 1]",0.0017,8,7


_________________________________________________
СКОп = 0.1, tau = 2, end_time = 1200
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.5 с
Показатели Монте-Карло:
Вариант 0: Рэф = 92%, Рэф_мин = 0
Вариант 1: Рэф = 98%, Рэф_мин = 0
Вариант 2: Рэф = 98%, Рэф_мин = 0
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.01
Nf = 9, Nf_max = 11
Iter = 8, Iter_max = 10
t = 0.0019, t_max = 0.0043


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t,Nf,Iter
0,0.0,40.000,98.786,11.58,360.0,40.185,98.291,11.70,22.92,38.282,22.94,38.597,0.01,"[1, 1, 1, 1]",0.0021,10,9
1,0.0,40.000,75.064,11.64,0.0,39.695,75.230,11.50,19.70,43.532,19.68,43.115,0.01,"[1, 1, 1, 1]",0.0021,10,9
2,0.0,40.000,78.479,3.31,0.0,40.217,83.726,3.25,7.56,38.512,7.57,38.361,0.01,"[0, 1, 1, 1]",0.0017,8,7
3,0.0,40.000,99.144,11.50,0.0,39.672,99.925,11.33,22.80,38.174,22.79,37.650,0.01,"[1, 1, 1, 1]",0.0020,10,9
4,0.0,40.000,174.065,9.57,0.0,40.061,174.317,9.77,5.19,26.072,5.17,25.880,0.01,"[1, 1, 1, 1]",0.0017,8,7


_________________________________________________
СКОп = 0.2, tau = 2, end_time = 1200
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.5 с
Показатели Монте-Карло:
Вариант 0: Рэф = 75%, Рэф_мин = 0
Вариант 1: Рэф = 93%, Рэф_мин = 0
Вариант 2: Рэф = 93%, Рэф_мин = 0
Вариант 3: Рэф = 96%, Рэф_мин = 0
Ка = 0.04
Nf = 9, Nf_max = 11
Iter = 8, Iter_max = 10
t = 0.0018, t_max = 0.0042


...

_________________________________________________
СКОп = 0.5, tau = 2, end_time = 1200
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.5 с
Показатели Монте-Карло:
Вариант 0: Рэф = 37%, Рэф_мин = 0
Вариант 1: Рэф = 63%, Рэф_мин = 0
Вариант 2: Рэф = 63%, Рэф_мин = 0
Вариант 3: Рэф = 81%, Рэф_мин = 0
Ка = 0.25
Nf = 9, Nf_max = 11
Iter = 8, Iter_max = 10
t = 0.0018, t_max = 0.0027


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t,Nf,Iter
0,0.0,40.000,98.786,11.58,359.9,40.946,96.378,12.17,22.92,38.282,23.03,39.887,0.25,"[0, 1, 1, 1]",0.0021,10,9
1,0.0,40.000,75.064,11.64,0.0,38.532,75.903,10.96,19.70,43.532,19.60,41.522,0.25,"[0, 1, 1, 1]",0.0026,10,9
2,0.0,40.000,78.479,3.31,0.1,41.077,105.537,3.29,7.56,38.512,7.64,37.738,0.26,"[0, 0, 0, 0]",0.0017,8,7
3,0.0,40.000,99.144,11.50,0.1,38.424,103.127,10.73,22.80,38.174,22.74,35.656,0.26,"[0, 1, 1, 1]",0.0020,10,9
4,0.0,40.000,174.065,9.57,0.0,40.309,175.216,10.60,5.19,26.072,5.10,25.119,0.24,"[0, 0, 0, 1]",0.0016,8,7


_________________________________________________
СКОп = 1.0, tau = 2, end_time = 1200
Алгоритм:: ММП v2
Моделирование 100 результатов закончено за t = 0.5 с
Показатели Монте-Карло:
Вариант 0: Рэф = 14%, Рэф_мин = 0
Вариант 1: Рэф = 38%, Рэф_мин = 0
Вариант 2: Рэф = 40%, Рэф_мин = 0
Вариант 3: Рэф = 49%, Рэф_мин = 0
Ка = 0.99
Nf = 9, Nf_max = 11
Iter = 8, Iter_max = 10
t = 0.0019, t_max = 0.0081


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t,Nf,Iter
0,0.0,40.000,98.786,11.58,359.8,41.941,94.136,12.82,22.92,38.282,23.15,41.562,1.00,"[0, 0, 0, 1]",0.0021,10,9
1,0.0,40.000,75.064,11.64,0.0,37.190,76.765,10.33,19.70,43.532,19.50,39.687,0.99,"[0, 0, 0, 1]",0.0022,10,9
2,0.0,40.000,78.479,3.31,0.1,42.136,128.476,3.94,7.56,38.512,7.73,36.911,1.05,"[0, 0, 0, 0]",0.0017,8,7
3,0.0,40.000,99.144,11.50,0.2,36.997,107.292,10.08,22.80,38.174,22.68,33.366,1.04,"[0, 0, 0, 1]",0.0022,10,9
4,0.0,40.000,174.065,9.57,0.0,40.628,176.140,11.62,5.19,26.072,5.02,24.188,0.94,"[0, 0, 0, 0]",0.0018,8,7


_________________________________________________


In [6]:
for std in std_arr:

    tma = TMA(observer, std=np.radians(std), tau=2, verbose=True)
    algorithm_name = 'Метод N пеленгов'
    res = [algorithm_name]
    dict_results = tma.swarm(n=100, algorithm_name=algorithm_name, target_func=target_func, p0=p0, seeded=True, verbose=True)
    df = tests.get_df(dict_results).dropna(axis=1)

    print('Показатели Монте-Карло:')
    for i in range(4):
        success = df['Успех'].apply(lambda x: x[i])
        success_rate = 100 * success.mean()
        success_min = min(success)
        print('Вариант {}: Рэф = {:.0f}%, Рэф_мин = {:.0f}'.format(i, success_rate, success_min))
        res.extend([success_rate, success_min])
    k_a_mean = df['Ка'].mean()
    t_mean = df['t'].mean()
    t_max = max(df['t'])
    std = np.degrees(tma.standart_deviation)
    print('Ка = {:.2f}'.format(k_a_mean))
    print('t = {:.4f}, t_max = {:.4f}'.format(t_mean, t_max))
    
    d_df = df.copy()
    for i, j in enumerate(df.columns):
        try: d_df[j] = df[j].apply(lambda x: precisions_1[i].format(np.float64(x)))
        except(TypeError): pass
    display(d_df.drop(['Кб'], axis=1).head())

    res.extend([k_a_mean, t_mean, t_max])
    results.append(res)
    tests.save_df(df, name='../tests/test1/d40/N, ' + str(np.degrees(tma.standart_deviation)) + '.xlsx')
    print('_'*40)

df_n = pd.DataFrame(results, columns=['Алгоритм', 'В11', 'В12', 'В21', 'В22', 'В31', 'В32', 'В41', 'В42', 'Ка', 't', 't_max'])
df_n['СКО'] = std_arr

СКОп = 0.0, tau = 2, end_time = 1200
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.4 с
Показатели Монте-Карло:
Вариант 0: Рэф = 100%, Рэф_мин = 1
Вариант 1: Рэф = 100%, Рэф_мин = 1
Вариант 2: Рэф = 100%, Рэф_мин = 1
Вариант 3: Рэф = 100%, Рэф_мин = 1
Ка = 0.00
t = 0.0002, t_max = 0.0014


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t
0,0.0,40.000,98.786,11.58,0.0,40.000,98.786,11.58,22.9,38.282,22.916,38.28,0.00,"[1, 1, 1, 1]",0.000
1,0.0,40.000,75.064,11.64,0.0,40.000,75.064,11.64,19.7,43.532,19.695,43.53,0.00,"[1, 1, 1, 1]",0.000
2,0.0,40.000,78.479,3.31,0.0,40.000,78.479,3.31,7.6,38.512,7.555,38.51,0.00,"[1, 1, 1, 1]",0.000
3,0.0,40.000,99.144,11.50,0.0,40.000,99.144,11.50,22.8,38.174,22.799,38.17,0.00,"[1, 1, 1, 1]",0.000
4,0.0,40.000,174.065,9.57,0.0,40.000,174.065,9.57,5.2,26.072,5.188,26.07,0.00,"[1, 1, 1, 1]",0.000


_________________________________________________
СКОп = 0.1, tau = 2, end_time = 1200
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.3 с
Показатели Монте-Карло:
Вариант 0: Рэф = 50%, Рэф_мин = 0
Вариант 1: Рэф = 67%, Рэф_мин = 0
Вариант 2: Рэф = 68%, Рэф_мин = 0
Вариант 3: Рэф = 71%, Рэф_мин = 0
Ка = 0.01
t = 0.0002, t_max = 0.0007


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t
0,0.0,40.000,98.786,11.58,359.8,40.440,94.578,12.09,22.9,38.282,22.925,39.81,0.01,"[1, 1, 1, 1]",0.000
1,0.0,40.000,75.064,11.64,359.8,39.720,71.397,12.19,19.7,43.532,19.661,44.36,0.01,"[1, 1, 1, 1]",0.000
2,0.0,40.000,78.479,3.31,0.0,39.627,94.583,3.06,7.6,38.512,7.530,37.04,0.01,"[0, 0, 0, 0]",0.000
3,0.0,40.000,99.144,11.50,359.8,40.154,94.837,11.88,22.8,38.174,22.776,39.41,0.01,"[1, 1, 1, 1]",0.000
4,0.0,40.000,174.065,9.57,360.0,39.660,174.418,9.66,5.2,26.072,5.141,25.61,0.01,"[1, 1, 1, 1]",0.000


_________________________________________________
СКОп = 0.2, tau = 2, end_time = 1200
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.3 с
Показатели Монте-Карло:
Вариант 0: Рэф = 6%, Рэф_мин = 0
Вариант 1: Рэф = 45%, Рэф_мин = 0
Вариант 2: Рэф = 46%, Рэф_мин = 0
Вариант 3: Рэф = 53%, Рэф_мин = 0
Ка = 0.05
t = 0.0002, t_max = 0.0005


...

_________________________________________________
СКОп = 0.5, tau = 2, end_time = 1200
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.3 с
Показатели Монте-Карло:
Вариант 0: Рэф = 0%, Рэф_мин = 0
Вариант 1: Рэф = 0%, Рэф_мин = 0
Вариант 2: Рэф = 0%, Рэф_мин = 0
Вариант 3: Рэф = 0%, Рэф_мин = 0
Ка = 0.44
t = 0.0002, t_max = 0.0016


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t
0,0.0,40.000,98.786,11.58,359.1,25.291,84.197,7.30,22.9,38.282,21.944,25.40,0.60,"[0, 0, 0, 0]",0.000
1,0.0,40.000,75.064,11.64,359.2,24.214,61.008,7.54,19.7,43.532,18.600,27.41,0.57,"[0, 0, 0, 0]",0.000
2,0.0,40.000,78.479,3.31,0.2,30.381,167.817,4.98,7.6,38.512,6.618,22.07,0.40,"[0, 0, 0, 0]",0.000
3,0.0,40.000,99.144,11.50,359.1,23.784,84.094,6.60,22.8,38.174,21.535,23.63,0.72,"[0, 0, 0, 0]",0.002
4,0.0,40.000,174.065,9.57,360.0,32.813,179.029,9.81,5.2,26.072,4.207,18.48,0.32,"[0, 0, 0, 0]",0.000


_________________________________________________
СКОп = 1.0, tau = 2, end_time = 1200
Алгоритм:: Метод N пеленгов
Моделирование 100 результатов закончено за t = 0.3 с
Показатели Монте-Карло:
Вариант 0: Рэф = 0%, Рэф_мин = 0
Вариант 1: Рэф = 0%, Рэф_мин = 0
Вариант 2: Рэф = 0%, Рэф_мин = 0
Вариант 3: Рэф = 0%, Рэф_мин = 0
Ка = 29.85
t = 0.0002, t_max = 0.0004


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,Ка,Успех,t
0,0.0,40.000,98.786,11.58,358.2,9.230,75.029,0.91,22.9,38.282,15.697,7.16,10.65,"[0, 0, 0, 0]",0.000
1,0.0,40.000,75.064,11.64,358.4,10.055,38.619,1.86,19.7,43.532,13.943,9.46,7.24,"[0, 0, 0, 0]",0.000
2,0.0,40.000,78.479,3.31,0.4,19.554,186.616,7.97,7.6,38.512,1.621,7.44,4.24,"[0, 0, 0, 0]",0.000
3,0.0,40.000,99.144,11.50,358.2,8.814,66.607,0.75,22.8,38.174,14.748,6.78,11.88,"[0, 0, 0, 0]",0.000
4,0.0,40.000,174.065,9.57,359.9,22.219,185.789,9.01,5.2,26.072,0.388,8.84,2.74,"[0, 0, 0, 0]",0.000


_________________________________________________


In [7]:
df = pd.concat([df_lm, df_n])
df['Ка'] = df['Ка'].apply(lambda x: np.sqrt(np.float64(x)))
df = df.rename(columns={'Ка' : 'sqrt(Ка)'})
for i, j in enumerate(df.columns):
        try: df[j] = df[j].apply(lambda x: precisions_3[i].format(np.float64(x)))
        except(ValueError): pass

print('Сводная таблица:')
df = df.groupby('СКО').apply(lambda a: a.drop('СКО', axis=1)[:]).reset_index().drop('level_1', axis=1)
df

Сводная таблица:


,СКО,Алгоритм,В11,В12,В21,В22,В31,В32,В41,В42,sqrt(Ка),Nf,Nf_max,Iter,Iter_max,t,t_max
0,0.0,ММП v2,100,1,100,1,100,1,100,1,0.00,9,11,8,10,0.0019,0.0043
1,0.0,Метод N пеленгов,100,1,100,1,100,1,100,1,0.00,nan,nan,nan,nan,0.0002,0.0014
2,0.1,ММП v2,92,0,98,0,98,0,100,1,0.10,9,11,8,10,0.0019,0.0043
3,0.1,Метод N пеленгов,50,0,67,0,68,0,71,0,0.11,nan,nan,nan,nan,0.0002,0.0007
4,0.2,ММП v2,75,0,93,0,93,0,96,0,0.20,9,11,8,10,0.0018,0.0042
5,0.2,Метод N пеленгов,6,0,45,0,46,0,53,0,0.22,nan,nan,nan,nan,0.0002,0.0005
6,0.3,ММП v2,55,0,86,0,86,0,92,0,0.30,9,11,8,10,0.0018,0.0032
7,0.3,Метод N пеленгов,0,0,1,0,10,0,24,0,0.34,nan,nan,nan,nan,0.0002,0.0007
8,0.5,ММП v2,37,0,63,0,63,0,81,0,0.50,9,11,8,10,0.0018,0.0027
9,0.5,Метод N пеленгов,0,0,0,0,0,0,0,0,0.66,nan,nan,nan,nan,0.0002,0.0016


In [8]:
import docx
doc = docx.Document()
t = doc.add_table(df.shape[0]+1, df.shape[1])
for j in range(df.shape[-1]):
    t.cell(0,j).text = df.columns[j]
for i in range(df.shape[0]):
    for j in range(df.shape[-1]):
        t.cell(i+1,j).text = str(df.values[i,j])
doc.save('../tests/test1/d40/Сводная таблица.docx')